In [1]:
import pandas as pd
import FinanceDataReader as fdr
import yfinance as yf
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

import yfinance as yf
import pandas as pd
import numpy as np
from yahoo_fin import stock_info as si
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.preprocessing import StandardScaler
from utils.prepro import preprocess_stock
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

### Get dataset list

1. We collect the dataset from FDR package 
2. We drop the non stock such as "fund"

In [2]:
kospi_symbol = fdr.StockListing('KOSPI')
nasdaq_symbol = fdr.StockListing('NASDAQ')
nyse_symbol = fdr.StockListing('NYSE')

sse_stocks = fdr.StockListing('SSE')   #상해거래소
HKEX_stocks = fdr.StockListing('HKEX') #홍콩거래소
TSE_stocks = fdr.StockListing('TSE')   #도쿄거래소
SZSE_stocks = fdr.StockListing('SZSE') #산전거래소

nasdaq_symbol["exchange"] = "NASDAQ"
sse_stocks["exchange"] = "SSE"
TSE_stocks["exchange"] = "TSE"
kospi_symbol["exchange"] = "KOSPI"
HKEX_stocks["exchange"] = "HKEX"
SZSE_stocks["exchange"] = "SZSE"
nyse_symbol["exchange"] = "NYES"

100% 4171/4171 [00:11<00:00, 354.88it/s]
100% 3057/3057 [00:08<00:00, 356.23it/s]
100% 1484/1484 [00:02<00:00, 711.39it/s]
100% 2695/2695 [00:06<00:00, 427.24it/s]
100% 4003/4003 [00:10<00:00, 375.35it/s]
100% 1751/1751 [00:04<00:00, 379.36it/s]


In [3]:
nasdaq_symbol = nasdaq_symbol[~nasdaq_symbol["Industry"].str.contains("펀드", na=False)]
sse_stocks = sse_stocks[~sse_stocks["Industry"].str.contains("펀드", na=False)]
TSE_stocks = TSE_stocks[~TSE_stocks["Industry"].str.contains("펀드", na=False)]
HKEX_stocks = HKEX_stocks[~HKEX_stocks["Industry"].str.contains("펀드", na=False)]
SZSE_stocks = SZSE_stocks[~SZSE_stocks["Industry"].str.contains("펀드", na=False)]
nyse_symbol = nyse_symbol[~nyse_symbol["Industry"].str.contains("펀드", na=False)]

In [4]:
kospi_symbol["Symbol"] = kospi_symbol["Code"] 

In [5]:
dacon_info = pd.read_csv("dacon_info.csv", encoding="CP949").iloc[:, :2]

In [6]:
len(set(nasdaq_symbol["Symbol"]).intersection(set(dacon_info['tck_iem_cd'].str.strip())))

2667

In [7]:
non_refer_stock = set(dacon_info["tck_iem_cd"].str.strip()).difference(set(nasdaq_symbol['Symbol'].str.strip()))

In [8]:
stock_list = pd.concat([nasdaq_symbol, nyse_symbol]).dropna().reset_index(drop = True)
stock_list = stock_list.drop("Name", axis = 1)
stock_list = stock_list.drop_duplicates().reset_index(drop = True)

### Get dataframe

In [10]:
# Notice that : 2023년 1월 1일 부터 2023년 8월 31일까지 발행된 정보(2023년 8월 31일 기준)

def fetch_data(symbol, exchange, industry_code):
    try:
        if exchange == "TSE":
            symbol = symbol + ".T"
        elif exchange == "SZSE":
            symbol = symbol + ".SZ"
        elif exchange == "SSE":
            symbol = symbol + ".SS"
        elif exchange == "KOSPI":
            symbol = symbol + ".KS"    
        
        #df = pdr.get_data_yahoo(symbol, start="2018-01-01",end = "2023-08-31", interval="1d").reset_index()[["Date", "Open","High","Low","Adj Close","Volume"]]
        ticker = yf.Ticker(symbol)
        df = ticker.history(start="2018-01-01",end = "2023-09-01", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]   
        df = preprocess_stock(df)
        df["Stock_"] = symbol
        df["IndustryCode_"] = industry_code
        #df["Name_"] = name
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None
    
    return df

def get_all_data(stock_list):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(fetch_data, symbol, exchange, industry_code) for symbol, exchange, industry_code in zip(stock_list["Symbol"], stock_list["exchange"], stock_list["IndustryCode"])]
        results = [future.result() for future in as_completed(futures)]

    data = pd.concat([result for result in results if result is not None])

    return data

In [11]:
nasdaq_data = get_all_data(stock_list)
TSE_data = get_all_data(TSE_stocks)
SZSE_data = get_all_data(SZSE_stocks)
sse_data = get_all_data(sse_stocks)

ARM: Data doesn't exist for startDate = 1514782800, endDate = 1693540800
Got error from yahoo api for ticker PPD, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- PPD: No timezone found, symbol may be delisted
Got error from yahoo api for ticker UHALB, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- UHALB: No timezone found, symbol may be delisted
Got error from yahoo api for ticker CREE, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- CREE: No timezone found, symbol may be delisted
Got error from yahoo api for ticker PBCT, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- PBCT: No timezone found, symbol may be delisted
CCG: Data doesn't exist for startDate = 1514782800, endDate = 1693540800
Got error from yahoo api for ticker ROLL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- ROLL: No timezone

### Data Cleaning

In [12]:
columns = ['Open06', 'Open12', 'Open18', 'Open24', 'Open30',
       'High06', 'High12', 'High18', 'High24', 'High30', 'Low06', 'Low12',
       'Low18', 'Low24', 'Low30', 'Close06', 'Close12', 'Close18', 'Close24',
       'Close30', 'Volume06', 'Volume12', 'Volume18', 'Volume24', 'Volume30']

def normalize_data(data):
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data)
    return normalized_data

def get_check_data(data):
    data_length = data["Stock_"].value_counts().reset_index()
    data = data[data["Stock_"].isin(data_length[data_length["Stock_"] >= 1000].iloc[:, 0].values)].reset_index(drop = True)
    data["Date"] = data["Date"].astype(str)
    date_ = [data["Date"][i][:10] for i in range(data.shape[0])]
    data["Date"] = date_
    data[columns] = normalize_data(data[columns])
    
    return data

In [13]:
nasdaq_data = get_check_data(nasdaq_data)
TSE_data = get_check_data(TSE_data)
SZSE_data = get_check_data(SZSE_data)
sse_data = get_check_data(sse_data)

In [14]:
all_sector_ = set(nasdaq_data["IndustryCode_"].unique()).union((TSE_data["IndustryCode_"].unique())).union(set(SZSE_data["IndustryCode_"].unique())).union(set(sse_data["IndustryCode_"].unique()))
unique_all_sector_ = list(set(list(all_sector_)))

In [15]:
import copy
unique_all_sector_sub = list(set(list(all_sector_))).copy()

In [30]:
len(unique_all_sector_sub)

137

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
sector_encoder = LabelEncoder()
sector_encoder.fit(unique_all_sector_)

nasdaq_data["IndustryCode_"] = sector_encoder.transform(nasdaq_data["IndustryCode_"])
TSE_data["IndustryCode_"] = sector_encoder.transform(TSE_data["IndustryCode_"])
SZSE_data["IndustryCode_"] = sector_encoder.transform(SZSE_data["IndustryCode_"])
sse_data["IndustryCode_"] = sector_encoder.transform(sse_data["IndustryCode_"])

In [18]:
def date_encoder_fun(data):
    date_encoder = LabelEncoder()
    date_encoder.fit(data["Date"].unique())
    data["Date_index"] = date_encoder.transform(data["Date"])
    return data

In [19]:
nasdaq_data = date_encoder_fun(nasdaq_data)
TSE_data = date_encoder_fun(TSE_data)
SZSE_data = date_encoder_fun(SZSE_data)
sse_data = date_encoder_fun(sse_data)

In [20]:
def get_train_test_split(data, is_save = False, name = None):
    train = data[data["Date"] <= "2023-01-01"].reset_index(drop = True)
    test = data[data["Date"] > "2023-01-01"].reset_index(drop = True)
    if is_save:
        train.to_csv("./data/train_{}.csv".format(name), index = False)
        test.to_csv("./data/test_{}.csv".format(name), index = False)
    else:
        return train, test

In [21]:
get_train_test_split(nasdaq_data, is_save = True, name = "nasdaq")
get_train_test_split(TSE_data, is_save = True, name = "tse")
get_train_test_split(SZSE_data, is_save = True,name =  "szse")
get_train_test_split(sse_data, is_save = True, name = "sse")

### Get index traking data

In [22]:
nasdaq_symbol = fdr.StockListing('NASDAQ')
nyse_symbol = fdr.StockListing('NYSE')
SZSE_stocks = fdr.StockListing('SZSE')
sse_stocks = fdr.StockListing('SSE')   # 상해거래소
HKEX_stocks = fdr.StockListing('HKEX') # 홍콩거래소
TSE_stocks = fdr.StockListing('TSE')   # 도쿄거래소

test_ind =pd.concat([nasdaq_symbol, nyse_symbol, TSE_stocks, SZSE_stocks,sse_stocks, HKEX_stocks]).reset_index(drop = True)

100% 4171/4171 [00:12<00:00, 334.60it/s]
100% 3057/3057 [00:09<00:00, 310.32it/s]
100% 1751/1751 [00:03<00:00, 514.58it/s]
100% 1484/1484 [00:02<00:00, 578.43it/s]
100% 2695/2695 [00:04<00:00, 562.99it/s]
100% 4003/4003 [00:07<00:00, 547.86it/s]


In [23]:
ticker = yf.Ticker("ARKK")
df = ticker.history(start="2023-01-01",end = "2023-09-01", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]  
df = preprocess_stock(df)

df['Date'] = pd.to_datetime(df['Date'])
df["Date"] = df["Date"].astype(str)
date_ = [df["Date"][i][:10] for i in range(df.shape[0])]
df["Date"] = date_

stocks = ["TSLA", "ZM", "ROKU", "PATH", "COIN", "SQ", "EXAX", "SHOP", "DKNG", "CRSP",
          "U", "TWLO", "TDOC", "PBLX", "NTLA", "BEAM", "PACB", "DNA", "HOOD", "PD",
          "TXG", "VCYT", "PLTR", "TWST", "CERS", "VERV", "TWOU", "NVTA"]

df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()
df7 = df.copy()
df8 = df.copy()
df9 = df.copy()
df10 = df.copy()
df11 = df.copy()
df12 = df.copy()
df13 = df.copy()

df1["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[0]
df2["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[1]
df3["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[2]
df4["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[3]
df5["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[4]
df6["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[5]
df7["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[6]
df8["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[7]
df9["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[8]
df10["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[9]
df11["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[10]
df12["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[11]
df13["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[12]
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13]
concatenated_df = pd.concat(df_list)

#concatenated_df["Date_index"] = date_encoder.fit_transform(concatenated_df["Date"])
concatenated_df["Sector"] = sector_encoder.transform(concatenated_df["IndustryCode_"])
concatenated_df[columns] = normalize_data(concatenated_df[columns])

concatenated_df.to_csv("./data/ARKK.csv", index = False)

In [25]:
ticker = yf.Ticker("SKYY")
df = ticker.history(start="2023-01-01",end = "2023-09-01", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]  
df = preprocess_stock(df)

df['Date'] = pd.to_datetime(df['Date'])
df["Date"] = df["Date"].astype(str)
date_ = [df["Date"][i][:10] for i in range(df.shape[0])]
df["Date"] = date_

stocks = ["MDB", "GOOGL", "MSFT", "AMZN", "OPCL", "PSTG", "ANET", "IBM", "APP", "AKAM",
          "NET", "SHOP", "CFLT", "CSCO", "HUBS", "ADBE", "DOCN", "NOW", "CRM", "TEAM",
          "WK", "NTAP", "RNG", "HCP", "SPLK", "TWLO", "HPE", "LUMN", "FSLY", "SAP",
          "APPN", "WIX", "ASAN", "TTD", "ZUO", "BLKB", "DDOG", "CRWD", "SQSP", "ESTC",
          "CRWD", "OTEX", "PANW", "APPF", "DBX", "TOST", "SNOW", "VEEV", "WDAY", "SMAR",
          "ZS", "INTU", "QLYS", "PAYS", "XM", "PCTY", "DOCU", "ZM", "BOX", "QTWO",
          "BL", "EB", "EVBG", "SPT", "RXT"]

df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()
df7 = df.copy()
df8 = df.copy()
df9 = df.copy()
df10 = df.copy()


df1["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[0]
df2["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[1]
df3["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[2]
df4["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[3]
df5["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[4]
df6["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[5]
df7["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[6]
df8["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[7]
df9["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[8]
df10["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[9]

df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
concatenated_df = pd.concat(df_list)
#concatenated_df["Date_index"] = date_encoder.fit_transform(concatenated_df["Date"])
concatenated_df["Sector"] = sector_encoder.transform(concatenated_df["IndustryCode_"])
concatenated_df[columns] = normalize_data(concatenated_df[columns])

concatenated_df.to_csv("./data/SKYY.csv", index = False)

In [26]:
ticker = yf.Ticker("BOTZ")
df = ticker.history(start="2023-01-01",end = "2023-09-01", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]  
df = preprocess_stock(df)

df['Date'] = pd.to_datetime(df['Date'])
df["Date"] = df["Date"].astype(str)
date_ = [df["Date"][i][:10] for i in range(df.shape[0])]
df["Date"] = date_

stocks = ["NVDA", "ISRG", "DT", "6861.T", "6954.T", "6645.T", "6506.T", "6273.T", "6393.T",
          "CGNX", "PATH", "ATS", "PEGA", "AI", "JBT", "OMCL", "AVAV", "UPST", "SYM", "APPN",
          "PRCT", "PRO", "CRCN", "IPBT", "6104.JP", "3993.JP", "6258.JP", "SOUN", "TSP",
          "4259.JP", "BGRY", "7779.JP", "FARO"]

df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()
df7 = df.copy()
df8 = df.copy()
df9 = df.copy()
df10 = df.copy()


df1["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[0]
df2["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[1]
df3["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[2]
df4["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[3]
df5["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[4]
df6["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[5]
df7["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[6]
df8["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[7]
df9["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[8]
df10["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[9]

df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
concatenated_df = pd.concat(df_list)

#concatenated_df["Date_index"] = date_encoder.fit_transform(concatenated_df["Date"])
concatenated_df["Sector"] = sector_encoder.transform(concatenated_df["IndustryCode_"])
concatenated_df[columns] = normalize_data(concatenated_df[columns])



concatenated_df.to_csv("./data/BOTZ.csv", index = False)

In [28]:
ticker = yf.Ticker("LIT")
df = ticker.history(start="2023-01-01",end = "2023-09-01", interval="1d").reset_index()[["Date", "Open","High","Low","Close","Volume"]]  
df = preprocess_stock(df)

df['Date'] = pd.to_datetime(df['Date'])
df["Date"] = df["Date"].astype(str)
date_ = [df["Date"][i][:10] for i in range(df.shape[0])]
df["Date"] = date_

stocks = ['ALB', '6752.T', 'TSLA', '6762.T', '1211.HK', '300750.SZ', '300014.SZ', '002466.SZ', '002371.SZ', 'RIVN', 'SQM',
 '002460.SZ',
 '002709.SZ',
 '002812.SZ',
 'LTHM',
 '300450.SZ',
 'LCID',
 'ENS',
 '603659.SS',
 'LTR AU',
 '300037.SZ',
 '300207.SZ',
 '300073.SZ',
 'SGML',
 'QS',
 'ENVX',
 '5631.T',
 '6674.T',
 'FREY']


df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()
df5 = df.copy()
df6 = df.copy()



df1["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[0]
df2["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[1]
df3["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[2]
df4["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[3]
df5["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[4]
df6["IndustryCode_"] = list(set(test_ind[test_ind["Symbol"].isin(stocks)]["IndustryCode"]))[5]

df_list = [df1, df2, df3, df4, df5, df6]

concatenated_df = pd.concat(df_list)

#concatenated_df["Date_index"] = date_encoder.fit_transform(concatenated_df["Date"])
concatenated_df["Sector"] = sector_encoder.transform(concatenated_df["IndustryCode_"])
concatenated_df[columns] = normalize_data(concatenated_df[columns])

concatenated_df.to_csv("./data/LIT.csv", index = False)